In [ ]:
import tensorflow as tf

In [ ]:
import argparse

parser = argparse.ArgumentParser(description='Train or execute model to generate texture from picture')
parser.add_argument('--input-path', type=str,
                    default="./data/input/wood",
                    help='Path where input and target images are stored')
parser.add_argument('--output-path', type=str,
                    default="./data/output",
                    help='Path where generated images will be stored')
parser.add_argument('--checkpoints-path', type=str,
                    default="./data/checkpoints",
                    help='Path where generated checkpoints will be stored')
parser.add_argument('--hide-plots', action="store_true",
                    default=False,
                    help='Do not display plots of every ')
parser.add_argument('-f')

args = parser.parse_args()

# Configuration
INPUT_PATH = args.input_path
OUTPUT_PATH = args.output_path
CHECKPOINTS_PATH = args.checkpoints_path
DISPLAY_PLOTS = not args.hide_plots

In [ ]:
# Auxiliar Functions

def normalize(image):
    image = (image / 127.5) - 1
    return image

def load_image(id):
    input_img = tf.cast(tf.image.decode_png(tf.io.read_file(INPUT_PATH + "/" + id + "-edges.png"), channels=3), tf.float32)
    target_img = tf.cast(tf.image.decode_png(tf.io.read_file(INPUT_PATH + "/" + id + "-image.png"), channels=3), tf.float32)
    
    input_img = normalize(input_img)
    target_img = normalize(target_img)

    return input_img, target_img

In [ ]:
import numpy as np
import glob
import json

# Main

image_paths = glob.glob(INPUT_PATH + "/*image*")
data_size = len(image_paths)

ids = list(map(lambda i: "%.4d" % (i,), range(data_size)))

train_size = round(data_size * 0.80)

ids_rand = np.copy(ids)
np.random.shuffle(ids_rand)

train_ids = ids_rand[:train_size]
test_ids = ids_rand[train_size:]

train_tensors = list(map(lambda i: load_image(i), train_ids))
test_tensors= list(map(lambda i: load_image(i), test_ids))

image_size = load_image(ids_rand[0])[0].shape

In [ ]:
print(image_size)

In [ ]:
import matplotlib.pyplot as plt

img = load_image(ids_rand[0])[1]
img = (img + 1) / 2
plt.imshow(img)
plt.show()

In [ ]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_tensors,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([None, None, 3]), tf.TensorShape([None, None, 3]))
)
train_dataset = train_dataset.batch(1)

test_dataset = tf.data.Dataset.from_generator(
    lambda: test_tensors,
    output_types=(tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([None, None, 3]), tf.TensorShape([None, None, 3]))
)
test_dataset = test_dataset.batch(1)

In [ ]:
for input_img, target_img in train_dataset.take(1):
    print(input_img.shape)
    plt.imshow((input_img[0,...] + 1) / 2)
    plt.show()
    
    print(target_img.shape)
    plt.imshow((target_img[0,...] + 1) / 2)
    plt.show()

In [ ]:
from tensorflow.keras import *
from tensorflow.keras.layers import *

def downsampler(filters, apply_batch_normalization=True):
    
    result = Sequential()
    
    initializer = tf.random_normal_initializer(0, 0.02)
    
    result.add(Conv2D(
        filters,
        kernel_size = 4,
        strides = 2,
        padding = "same",
        kernel_initializer = initializer,
        use_bias = not apply_batch_normalization
    ))
    
    if apply_batch_normalization:
        result.add(BatchNormalization())
    
    result.add(LeakyReLU(alpha = 0.2))
    
    return result

In [ ]:
downsampler(64)

In [ ]:
def upsampler(filters, apply_dropout=False):
    result = Sequential()
    
    initializer = tf.random_normal_initializer(0, 0.02)
    
    result.add(Conv2DTranspose(
        filters,
        kernel_size = 4,
        strides = 2,
        padding = "same",
        kernel_initializer = initializer,
        use_bias = False
    ))
    
    result.add(BatchNormalization())
    
    if apply_dropout:
        result.add(Dropout(0.5))
    
    result.add(ReLU())
    
    return result

In [ ]:
upsampler(64)

In [ ]:
def Generator(input_dim1, input_dim2):
    inputs = tf.keras.layers.Input(shape=[input_dim1, input_dim2, 3])
    
    last_layer = Conv2DTranspose(
        filters = 3,
        kernel_size = 4,
        strides = 2,
        padding = "same",
        kernel_initializer = tf.random_normal_initializer(0, 0.02),
        activation = "tanh"
    )

    # Encoder
    l_e1 = downsampler(64, apply_batch_normalization = False)(inputs)
    l_e2 = downsampler(128)(l_e1)
    l_e3 = downsampler(256)(l_e2)
    l_e4 = downsampler(512)(l_e3)
    l_e5 = downsampler(512)(l_e4)
    l_e6 = downsampler(512)(l_e5)
    l_e7 = downsampler(512)(l_e6)
    l_e8 = downsampler(512)(l_e7)

    # Decoder
    l_d1 = upsampler(512, apply_dropout = True)(l_e8)
    l_d2 = upsampler(512, apply_dropout = True)(concatenate([l_d1, l_e7]))
    l_d3 = upsampler(512, apply_dropout = True)(concatenate([l_d2, l_e6]))
    l_d4 = upsampler(512)(concatenate([l_d3, l_e5]))
    l_d5 = upsampler(256)(concatenate([l_d4, l_e4]))
    l_d6 = upsampler(128)(concatenate([l_d5, l_e3]))
    l_d7 = upsampler(64)(concatenate([l_d6, l_e2]))
    
    last = last_layer(l_d7)
    
    return Model(inputs=[inputs], outputs=last)

generator = Generator(image_size[0], image_size[1])

In [ ]:
for input_img, target_img in train_dataset.take(1):
    gen_output = generator([((input_img + 1) * 255)], training=False)
    plt.imshow(gen_output[0,...])

In [ ]:
def Discriminator(input_dim1, input_dim2):
    input_img = Input(shape=[input_dim1, input_dim2, 3])
    generated_img = Input(shape=[input_dim1, input_dim2, 3])
                
    l_d1 = downsampler(64, apply_batch_normalization=False)(concatenate([input_img, generated_img]))
    l_d2 = downsampler(128)(l_d1)
    l_d3 = downsampler(256)(l_d2)
    l_d4 = downsampler(512)(l_d3)
        
    last = Conv2D(
        filters = 1,
        kernel_size = 4,
        strides = 2,
        padding = "same",
        kernel_initializer = tf.random_normal_initializer(0, 0.02),
    )(l_d4)
    
    return Model(inputs=[input_img, generated_img], outputs=last)

discriminator = Discriminator(image_size[0], image_size[1])

In [ ]:
for input_img, target_img in train_dataset.take(1):
    gen_output = generator([((input_img + 1) * 255)], training=False)
    disc_out = discriminator([((input_img + 1) * 255), gen_output], training=False)
    
    plt.imshow(disc_out[0,...,-1], vmin=-20, vmax=20, cmap='RdBu_r')
    plt.colorbar()

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
    
    total_loss = real_loss + generated_loss
    
    return total_loss

In [ ]:
LAMBDA = 100

def generator_loss(disc_generated_output, generated_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    
    l1_loss = tf.reduce_mean(tf.abs(target - generated_output))
    
    total_loss = gan_loss + (LAMBDA * l1_loss)
    
    return total_loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [ ]:
checkpoint_prefix = CHECKPOINTS_PATH + "/chkp"

checkpoint = tf.train.Checkpoint(
    generator_optimizer=generator_optimizer,
    discriminator_optimizer=discriminator_optimizer,
    generator=generator,
    discriminator=discriminator
)

In [ ]:
def generate_images(model, test_input, test_target, save_filename=False, display_imgs=True):
    prediction = model([test_input], training = False)
    
    if save_filename:
        tf.keras.preprocessing.image.save_img(OUTPUT_PATH + '/' + save_filename + ".jpg", prediction[0, ...])
        
    if display_imgs:
        plt.figure(figsize=(10,10))

        display_list = [test_input[0], test_target[0], prediction[0]]
        title = ["Input image", "Ground truth", "Predicted Image"]
    
        for i in range(3):
            plt.subplot(1, 3, i+1)
            plt.title(title[i])
            
            plt.imshow(display_list[i] * 0.5 + 0.5)
            plt.axis("off")
    
        plt.show()

In [ ]:
@tf.function()
def train_step(input_image, target_image):
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        
        generated_image = generator([input_image], training=True)
    
        generated_image_disc = discriminator([generated_image, input_image], training = True)
    
        target_image_disc = discriminator([target_image, input_image], training = True)
    
        disc_loss = discriminator_loss(target_image_disc, generated_image_disc)
    
        gen_loss = generator_loss(generated_image_disc, generated_image, target_image)
    
        discriminator_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        generator_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
        
        discriminator_optimizer.apply_gradients(zip(discriminator_grads, discriminator.trainable_variables))
        generator_optimizer.apply_gradients(zip(generator_grads, generator.trainable_variables))
        

In [ ]:
from IPython.display import clear_output
import os

def train(dataset, epochs):
    
    if os.listdir(CHECKPOINTS_PATH):
        checkpoint.restore(tf.train.latest_checkpoint(CHECKPOINTS_PATH)).assert_consumed()
    
    for epoch in range(epochs):
        img_counter = 0
        for input_image, target_image in train_dataset:
            print("epoch %d - train: %d / %d" % (epoch, img_counter, len(train_ids)))
            img_counter += 1
            train_step(input_image, target_image)
        
        clear_output(wait=True)

        img_counter = 0
        for input_image, target_image in test_dataset.take(5):
            generate_images(generator, input_image, target_image, "%d_%d" % (img_counter, epoch), display_imgs=DISPLAY_PLOTS)
            img_counter += 1
            
        if (epoch + 1) % 25 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
train(train_dataset, 1000)